In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install -U weaviate-client  # For beta versions: `pip install --pre -U "weaviate-client==4.*"`

Note: you may need to restart the kernel to use updated packages.


In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
import os
import weaviate
import PyPDF2
from transformers import AutoTokenizer, AutoModel

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/PyPDF2/__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(


In [9]:
# Carga el texto de documentos PDF
def cargar_texto_desde_pdf(ruta_pdf):
    texto = ""
    with open(ruta_pdf, "rb") as archivo_pdf:
        lector_pdf = PyPDF2.PdfReader(archivo_pdf)
        for pagina in lector_pdf.pages:
            texto += pagina.extract_text()
    return texto

ruta_pdf = "PDFs/02BD1_MDD_ppt.pdf"
texto_pdf = cargar_texto_desde_pdf(ruta_pdf)

print(texto_pdf)

1Base de Datos 1 
• Licenciatura e Ingeniería en Informática • 2do. año
Este material es de uso exclusivo para los cursos i mpartidos por Universidad de la Empresa y asociados 2
Este material es de uso exclusivo para los cursos i mpartidos por Universidad de la Empresa y asociados Modelos de Datos 
• Conceptos Generales 
–Interpretar una realidad –Definir reglas que permitan estructurar los datos –Estrictamente tipeados(cada dato pertenece a algun a categoría 
con sus propiedades). 
• Modelo de Datos 
M ( G , O ) 
–G  Reglas generatrices 
•Estructuras •Restricciones 
–O Operaciones 
•Dinámicas •Cambio en el estado de la BD 3
Este material es de uso exclusivo para los cursos i mpartidos por Universidad de la Empresa y asociados Estructuras 
• Abstracción
–Ocultar los detalles y concentrarse en los concept os comunes 
generales a un objeto –Obtener la categorización de datos 
• Niveles de abstracción
–Nivel de token 
•Valor real, instancia particular de un objeto. 
–Nivel de tipo 
•Clase

In [10]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nltk
nltk.download('punkt')  # Descarga los datos necesarios para tokenizar oraciones

from nltk.tokenize import sent_tokenize

def split_into_sentences(text):
    return sent_tokenize(text)

# Ejemplo de uso:
input_text = "Este es un ejemplo de texto. Contiene dos oraciones."
sentences = split_into_sentences(input_text)
print(sentences)


In [ ]:
# python src/transformers/models/llama/convert_llama_weights_to_hf.py \
#     --input_dir /path/to/downloaded/llama/weights --model_size 7B --output_dir /output/path

In [11]:
# Carga el modelo Llama-2
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)
model = AutoModel.from_pretrained(model_name)

# Carga el modelo Llama-2 preentrenado
# tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7B-Chat-GGML")
# model = AutoModel.from_pretrained("TheBloke/Llama-2-7B-Chat-GGML")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
import torch

In [ ]:
# # Tokeniza el texto
# tokenized_text = tokenizer(sentences, pad_token="[PAD]", padding=True, truncation=True, return_tensors="pt")

# # Genera representaciones vectoriales
# with torch.no_grad():
#     outputs = model(tokenized_text)

# # Las representaciones vectoriales están en outputs.last_hidden_state
# representations = outputs.last_hidden_state

In [13]:

# Configura la clave de API de Weaviate
os.environ['WEAVIATE_API_KEY'] = "TP3jGvfknJ3ZIQGDthvpWs3QkGtJaN8C7e0l"

In [14]:
# Consulta Weaviate para buscar documentos relevantes
def buscar_documentos(query):
    cliente_weaviate = weaviate.Client(
        url = "https://prueba1-dzfgg9kd.weaviate.network",  # Replace with your endpoint
        auth_client_secret=weaviate.auth.AuthApiKey(api_key="TP3jGvfknJ3ZIQGDthvpWs3QkGtJaN8C7e0l"),  # Replace w/ your Weaviate instance API key
    )
    resultados = cliente_weaviate.query.get(
        "DocumentosPDF",
        ['body']
    ).with_near_text(
        {"concepts": [query]}
    ).with_limit(5).do()
    return resultados

In [15]:
def almacenar_en_weaviate(representaciones_vectoriales):
    cliente_weaviate = weaviate.Client(
        url = "https://prueba1-dzfgg9kd.weaviate.network",  # Replace with your endpoint
        auth_client_secret = weaviate.auth.AuthApiKey(api_key="TP3jGvfknJ3ZIQGDthvpWs3QkGtJaN8C7e0l"),  # Replace w/ your Weaviate instance API key
    )
    clase_documentos_pdf = "DocumentosPDF"  # Crea esta clase en tu instancia de Weaviate

    # for idx, representacion in enumerate(representaciones_vectoriales):
    #     cliente_weaviate.data_object.create(
    #         class_name = clase_documentos_pdf,
    #         vector = representacion.tolist()
    #     )

    # for idx, representacion in enumerate(representaciones_vectoriales):
    # # Crea un objeto de datos con un vector
    #     data_object = {
    #         "name" : "MyVectorData",  # Nombre de la clase
    #         "myVectorProperty" : [0.1, 0.2, 0.3]  # Vector de ejemplo
    #     }
    #     # Guarda el objeto de datos en Weaviate
    #     cliente_weaviate.data.create(data_object, class_name="MyVectorData")

    for idx, representacion in enumerate(representaciones_vectoriales):
        cliente_weaviate.batch.add_data_object(
            {"source_text": representacion['texto'],},
            # source_text = representacion['texto'],
            # class_name = clase_documentos_pdf,
            clase_documentos_pdf,
            vector = representacion['vector'].tolist()
        )
    # cliente_weaviate.batch.add_data_object(
    #         # data_obj, 
    #         "MyVectorData", 
    #         vector=vector  # Provide openai vector
    #     )

    # for idx, representacion in enumerate(representaciones_vectoriales):
    #     cliente_weaviate.batch.create(
    #         clase_documentos_pdf,
    #         [
    #             {"id": f"documento_{idx}", "vector": representacion.tolist()}
    #         ]
    #     )

In [16]:
def generar_representaciones_vectoriales(texto):
    # Ejemplo ficticio:
    representaciones_vectoriales = []
    for oracion in texto.split("."):
        # Tokeniza y genera representación vectorial
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        entrada = tokenizer.encode(oracion, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            representacion = {
                "texto" : oracion,
                "vector" : model(entrada).last_hidden_state.mean(dim=1).numpy()
            }
            # representacion = model(entrada).last_hidden_state.mean(dim=1).numpy()
            representaciones_vectoriales.append(representacion)
    print(representaciones_vectoriales)
    return representaciones_vectoriales

In [17]:
client = weaviate.Client(
    url = "https://prueba1-dzfgg9kd.weaviate.network",  # Replace with your endpoint
    auth_client_secret=weaviate.auth.AuthApiKey(api_key="TP3jGvfknJ3ZIQGDthvpWs3QkGtJaN8C7e0l"),  # Replace w/ your Weaviate instance API key
)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


In [18]:
def jprint(json_in):
    import json
    print(json.dumps(json_in, indent=2))

jprint(client.get_meta())

{
  "hostname": "http://[::]:8080",
  "modules": {
    "generative-aws": {
      "documentationHref": "https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html",
      "name": "Generative Search - AWS"
    },
    "generative-cohere": {
      "documentationHref": "https://docs.cohere.com/reference/generate",
      "name": "Generative Search - Cohere"
    },
    "generative-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "Generative Search - OpenAI"
    },
    "generative-palm": {
      "documentationHref": "https://cloud.google.com/vertex-ai/docs/generative-ai/chat/test-chat-prompts",
      "name": "Generative Search - Google PaLM"
    },
    "qna-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "OpenAI Question & Answering Module"
    },
    "ref2vec-centroid": {},
    "reranker-cohere": {
      "documentationHref": "https://txt.cohere.com/reran

In [20]:
# jprint(client.schema.get())

class_obj = {
    "class": "DocumentosPDF2",
    "vectorizer": "text2vec-huggingface",
    "moduleConfig": {
        "text2vec-huggingface": {
            "vectorizeClassName": True
        }
    } 
}

# Add the class to the schema
client.schema.create_class(class_obj)

jprint(client.schema.get())

{
  "classes": [
    {
      "class": "DocumentosPDF",
      "invertedIndexConfig": {
        "bm25": {
          "b": 0.75,
          "k1": 1.2
        },
        "cleanupIntervalSeconds": 60,
        "stopwords": {
          "additions": null,
          "preset": "en",
          "removals": null
        }
      },
      "multiTenancyConfig": {
        "enabled": false
      },
      "properties": [],
      "replicationConfig": {
        "factor": 1
      },
      "shardingConfig": {
        "virtualPerPhysical": 128,
        "desiredCount": 1,
        "actualCount": 1,
        "desiredVirtualCount": 128,
        "actualVirtualCount": 128,
        "key": "_id",
        "strategy": "hash",
        "function": "murmur3"
      },
      "vectorIndexConfig": {
        "skip": false,
        "cleanupIntervalSeconds": 300,
        "maxConnections": 64,
        "efConstruction": 128,
        "ef": -1,
        "dynamicEfMin": 100,
        "dynamicEfMax": 500,
        "dynamicEfFactor": 8,
    

In [21]:
# Ejemplo de uso
if __name__ == "__main__":
    print("-- 1 --")
    ruta_pdf = "PDFs/02BD1_MDD_ppt.pdf"
    texto_pdf = cargar_texto_desde_pdf(ruta_pdf)
    representaciones = generar_representaciones_vectoriales(texto_pdf)
    print(representaciones)
    almacenar_en_weaviate(representaciones)

    consulta_usuario = "buscar información relevante"
    resultados_busqueda = buscar_documentos(consulta_usuario)
    print(resultados_busqueda)

-- 1 --
[{'texto': '1Base de Datos 1 \n• Licenciatura e Ingeniería en Informática • 2do', 'vector': array([[ 0.9307846 ,  0.22104606, -0.04142186, ...,  0.68621475,
         0.13347778,  0.59272456]], dtype=float32)}, {'texto': ' año\nEste material es de uso exclusivo para los cursos i mpartidos por Universidad de la Empresa y asociados 2\nEste material es de uso exclusivo para los cursos i mpartidos por Universidad de la Empresa y asociados Modelos de Datos \n• Conceptos Generales \n–Interpretar una realidad –Definir reglas que permitan estructurar los datos –Estrictamente tipeados(cada dato pertenece a algun a categoría \ncon sus propiedades)', 'vector': array([[ 6.09859943e-01, -3.44942957e-01,  1.16705894e-04, ...,
        -2.72184014e-01,  2.45567232e-01,  4.54548627e-01]], dtype=float32)}, {'texto': ' \n• Modelo de Datos \nM ( G , O ) \n–G  Reglas generatrices \n•Estructuras •Restricciones \n–O Operaciones \n•Dinámicas •Cambio en el estado de la BD 3\nEste material es de uso excl